## Using SQL with Python and Pandas via pandasql


### Use Cases
- Data fits in memory
- Volume of data not a performance issue
- Ease of using SQL
- Don't need a fully functional SQL Server database

### https://github.com/yhat/pandasql/tree/master/examples

In [1]:
from sklearn.datasets import load_iris
import pandas as pd
from pandasql import sqldf
from pandasql import load_meat, load_births
import re

births = load_births()
meat = load_meat()
iris = load_iris()
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df['species'] = pd.Categorical.from_codes(iris.target, iris.target_names)
iris_df.columns = [re.sub("[() ]", "", col) for col in iris_df.columns]

print(sqldf("SELECT * FROM iris_df LIMIT 10;", locals()))
print(sqldf("SELECT sepalwidthcm, species FROM iris_df LIMIT 10;", locals()))

   sepallengthcm  sepalwidthcm  petallengthcm  petalwidthcm species
0            5.1           3.5            1.4           0.2  setosa
1            4.9           3.0            1.4           0.2  setosa
2            4.7           3.2            1.3           0.2  setosa
3            4.6           3.1            1.5           0.2  setosa
4            5.0           3.6            1.4           0.2  setosa
5            5.4           3.9            1.7           0.4  setosa
6            4.6           3.4            1.4           0.3  setosa
7            5.0           3.4            1.5           0.2  setosa
8            4.4           2.9            1.4           0.2  setosa
9            4.9           3.1            1.5           0.1  setosa
   sepalwidthcm species
0           3.5  setosa
1           3.0  setosa
2           3.2  setosa
3           3.1  setosa
4           3.6  setosa
5           3.9  setosa
6           3.4  setosa
7           3.4  setosa
8           2.9  setosa
9           

In [2]:
q = """
      select
        species
        , avg(sepalwidthcm)
        , min(sepalwidthcm)
        , max(sepalwidthcm)
      from
        iris_df
      group by
        species;
        
"""
print("*" * 80)
print("aggregation")
print("-" * 80)
print(q)
print(sqldf(q, locals()))

********************************************************************************
aggregation
--------------------------------------------------------------------------------

      select
        species
        , avg(sepalwidthcm)
        , min(sepalwidthcm)
        , max(sepalwidthcm)
      from
        iris_df
      group by
        species;
        

      species  avg(sepalwidthcm)  min(sepalwidthcm)  max(sepalwidthcm)
0      setosa              3.418                2.3                4.4
1  versicolor              2.770                2.0                3.4
2   virginica              2.974                2.2                3.8


In [4]:
# Just to make calling pysqldf a bit easier...

def pysqldf(q):
    "add this to your script if you get tired of calling locals()"
    return sqldf(q, globals())

### Only inner joins are supported by pysqldf

In [5]:
q = """
    select
        a.*
    from
        iris_df a
    inner join
        iris_df b
            on a.species = b.species
    limit 10;
"""

print("*" * 80)
print("joins")
print("-" * 80)
print(q)
print(pysqldf(q))

********************************************************************************
joins
--------------------------------------------------------------------------------

    select
        a.*
    from
        iris_df a
    inner join
        iris_df b
            on a.species = b.species
    limit 10;

   sepallengthcm  sepalwidthcm  petallengthcm  petalwidthcm species
0            5.1           3.5            1.4           0.2  setosa
1            5.1           3.5            1.4           0.2  setosa
2            5.1           3.5            1.4           0.2  setosa
3            5.1           3.5            1.4           0.2  setosa
4            5.1           3.5            1.4           0.2  setosa
5            5.1           3.5            1.4           0.2  setosa
6            5.1           3.5            1.4           0.2  setosa
7            5.1           3.5            1.4           0.2  setosa
8            5.1           3.5            1.4           0.2  setosa
9            5.1

## Let's use our own data...

In [7]:
pwd

'D:\\DocumentsD\\Bryant\\PythonAnalytics\\General\\PythonSQLServer\\Notebook'

In [22]:
import pandas as pd

filepath = './Data/'

dfcustomer = pd.read_csv(filepath + 'DimCustomer.csv')

In [23]:
dfcustomer.head()

CustomerKey  GeographyKey CustomerAlternateKey Title  FirstName MiddleName  \
0        11000            26           AW00011000   NaN        Jon          V   
1        11001            37           AW00011001   NaN     Eugene          L   
2        11002            31           AW00011002   NaN      Ruben        NaN   
3        11003            11           AW00011003   NaN    Christy        NaN   
4        11004            19           AW00011004   NaN  Elizabeth        NaN   

  LastName  NameStyle   BirthDate MaritalStatus       ...        \
0     Yang          0  1971-10-06             M       ...         
1    Huang          0  1976-05-10             S       ...         
2   Torres          0  1971-02-09             M       ...         
3      Zhu          0  1973-08-14             S       ...         
4  Johnson          0  1979-08-05             S       ...         

  EnglishOccupation SpanishOccupation FrenchOccupation  HouseOwnerFlag  \
0      Professional       Profesional            Cadre               1   
1      Professional       Profesional            Cadre               0   
2      Professional       Profesional            Cadre               1   
3      Professional       Profesional            Cadre               0   
4      Professional       Profesional            Cadre               1   

   NumberCarsOwned         AddressLine1 AddressLine2                Phone  \
0                0      3761 N. 14th St          NaN  1 (11) 500 555-0162   
1                1           2243 W St.          NaN  1 (11) 500 555-0110   
2                1     5844 Linden Land          NaN  1 (11) 500 555-0184   
3                1     1825 Village Pl.          NaN  1 (11) 500 555-0162   
4                4  7553 Harness Circle          NaN  1 (11) 500 555-0131   

  DateFirstPurchase CommuteDistance  
0        2011-01-19       1-2 Miles  
1        2011-01-15       0-1 Miles  
2        2011-01-07       2-5 Miles  
3        2010-12-29      5-10 Miles  
4        2011-01-23       1-2 Miles  

[5 rows x 29 columns]

In [24]:
dfinternetsales = pd.read_csv(filepath + 'FactInternetSales.csv')
dfinternetsales.dtypes

ProductKey                 int64
OrderDateKey               int64
DueDateKey                 int64
ShipDateKey                int64
CustomerKey                int64
PromotionKey               int64
CurrencyKey                int64
SalesTerritoryKey          int64
SalesOrderNumber          object
SalesOrderLineNumber       int64
RevisionNumber             int64
OrderQuantity              int64
UnitPrice                float64
ExtendedAmount           float64
UnitPriceDiscountPct       int64
DiscountAmount             int64
ProductStandardCost      float64
TotalProductCost         float64
SalesAmount              float64
TaxAmt                   float64
Freight                  float64
CarrierTrackingNumber    float64
CustomerPONumber         float64
OrderDate                 object
DueDate                   object
ShipDate                  object
dtype: object

In [26]:
import pandas as pd
from pandasql import sqldf

print(pysqldf('select * from dfcustomer limit 5'))

   CustomerKey  GeographyKey CustomerAlternateKey Title  FirstName MiddleName  \
0        11000            26           AW00011000  None        Jon          V   
1        11001            37           AW00011001  None     Eugene          L   
2        11002            31           AW00011002  None      Ruben       None   
3        11003            11           AW00011003  None    Christy       None   
4        11004            19           AW00011004  None  Elizabeth       None   

  LastName  NameStyle   BirthDate MaritalStatus       ...        \
0     Yang          0  1971-10-06             M       ...         
1    Huang          0  1976-05-10             S       ...         
2   Torres          0  1971-02-09             M       ...         
3      Zhu          0  1973-08-14             S       ...         
4  Johnson          0  1979-08-05             S       ...         

  EnglishOccupation SpanishOccupation FrenchOccupation  HouseOwnerFlag  \
0      Professional       Profesiona

In [28]:
#  Joining dataframes with SQL

query = '''select * from dfcustomer c
            join dfinternetsales     s
            on (c.CustomerKey = s.CustomerKey) limit 5'''

pysqldf(query)

CustomerKey  GeographyKey CustomerAlternateKey Title FirstName MiddleName  \
0        11000            26           AW00011000  None       Jon          V   
1        11000            26           AW00011000  None       Jon          V   
2        11000            26           AW00011000  None       Jon          V   
3        11000            26           AW00011000  None       Jon          V   
4        11000            26           AW00011000  None       Jon          V   

  LastName  NameStyle   BirthDate MaritalStatus           ...             \
0     Yang          0  1971-10-06             M           ...              
1     Yang          0  1971-10-06             M           ...              
2     Yang          0  1971-10-06             M           ...              
3     Yang          0  1971-10-06             M           ...              
4     Yang          0  1971-10-06             M           ...              

  ProductStandardCost TotalProductCost SalesAmount    TaxAmt  Freight  \
0             13.0863          13.0863       34.99    2.7992   0.8748   
1           1912.1544        1912.1544     3399.99  271.9992  84.9998   
2           1265.6195        1265.6195     2319.99  185.5992  57.9998   
3              8.2205           8.2205       21.98    1.7584   0.5495   
4             41.5723          41.5723       53.99    4.3192   1.3498   

   CarrierTrackingNumber CustomerPONumber                OrderDate  \
0                   None             None  2013-05-03 00:00:00.000   
1                   None             None  2011-01-19 00:00:00.000   
2                   None             None  2013-01-18 00:00:00.000   
3                   None             None  2013-01-18 00:00:00.000   
4                   None             None  2013-05-03 00:00:00.000   

                   DueDate                 ShipDate  
0  2013-05-15 00:00:00.000  2013-05-10 00:00:00.000  
1  2011-01-31 00:00:00.000  2011-01-26 00:00:00.000  
2  2013-01-30 00:00:00.000  2013-01-25 00:00:00.000  
3  2013-01-30 00:00:00.000  2013-01-25 00:00:00.000  
4  2013-05-15 00:00:00.000  2013-05-10 00:00:00.000  

[5 rows x 55 columns]